In [4]:
import numpy as np
import torch
from torchvision import models
from sklearn.model_selection import StratifiedShuffleSplit
from data_utils import *
%load_ext autoreload
%autoreload 2
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
## Load the data
data = load_bolete_data()

ItemsViewHDF5(<HDF5 file "bolete.h5" (mode r)>)


In [6]:
# Get Train and Test Split
X_train, X_test, Y_train, Y_test = get_train_and_test(data, 'bolete-characteristics')


In [7]:
## Download a pretrained model
googlenet = models.googlenet(pretrained=True)


In [10]:
## Transfer Learn on new data
## inspired by https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

import torch.optim as optim
import torch.nn as nn
from train import cross_val

dtype = torch.cuda.FloatTensor

def loss_fn(scores, true_y):
    return sum(scores == true_y)

num_classes = Y_train.shape[1]
googlenet.fc = nn.Linear(512, num_classes)
optimizer = optim.Adam(googlenet.parameters(), betas=(.5, .999), lr=1e-3)

cross_val(
    X_train, 
    Y_train, 
    googlenet, 
    optimizer, 
    loss_fn, 
    64, 
    10,
    5,
    device)

# cross_val_split = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
# cross_val_split.get_n_splits(X_train, Y_train)
# for train_index, val_index in cross_val_split.split(X_train, Y_train):
#     X_train_cv, X_val_cv, Y_train_cv, Y_val_cv = get_data_from_splits(train_index, val_index, 'bolete-characteristics')
#     train_loader = get_loader(X_train_cv, Y_train_cv)
#     val_loader = get_loader(X_val_cv, Y_val_cv)
#     dataloaders = {'train':train_loader, 'val': val_loader}
#     train_model(
#         model=googlenet, 
#         dataloaders = dataloaders, 
#         loss_fn = loss_fn, 
#         optimizer=optimizer, 
#         num_epochs=20, 
#         show_every=5
#         )


TypeError: get_loader() missing 2 required positional arguments: 'batch_size' and 'shuffle'

In [84]:
cross_val()

RuntimeError: Given input size: (192x128x1). Calculated output size: (192x64x0). Output size is too small